<a href="https://colab.research.google.com/github/rpdahxn/GenerativeDeepLearning/blob/main/03_VAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 3.2.1 첫 번째 AE

In [26]:
AE = Autoencoder(
    input_dim = (28, 28, 1),
    encoder_conv_filters = [32, 64, 64, 64],
    encoder_conv_kernel_size = [3, 3, 3, 3],
    encoder_conv_strides = [1, 2, 2, 1],  # stride = 2로 출력의 크기를 줄인다.
    decoder_conv_t_filters = [64, 64, 32, 1],
    decoder_conv_t_kernel_size = [3, 3, 3, 3],
    decoder_conv_t_strides = [1, 2, 2, 1],
    z_dim = 2
)

In [27]:
AE.encoder.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, 28, 28, 1)]       0         
                                                                 
 encoder_conv_0 (Conv2D)     (None, 28, 28, 32)        320       
                                                                 
 leaky_re_lu_14 (LeakyReLU)  (None, 28, 28, 32)        0         
                                                                 
 encoder_conv_1 (Conv2D)     (None, 14, 14, 64)        18496     
                                                                 
 leaky_re_lu_15 (LeakyReLU)  (None, 14, 14, 64)        0         
                                                                 
 encoder_conv_2 (Conv2D)     (None, 7, 7, 64)          36928     
                                                                 
 leaky_re_lu_16 (LeakyReLU)  (None, 7, 7, 64)          0   

In [28]:
AE.decoder.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 decoder_input (InputLayer)  [(None, 2)]               0         
                                                                 
 dense_2 (Dense)             (None, 3136)              9408      
                                                                 
 reshape_2 (Reshape)         (None, 7, 7, 64)          0         
                                                                 
 decoder_conv_t_0 (Conv2DTra  (None, 7, 7, 64)         36928     
 nspose)                                                         
                                                                 
 leaky_re_lu_18 (LeakyReLU)  (None, 7, 7, 64)          0         
                                                                 
 decoder_conv_t_1 (Conv2DTra  (None, 14, 14, 64)       36928     
 nspose)                                                   

### 3.2.2 인코더
Autoencoder 클래스 중 인코더 부분

In [ ]:
# 인코더의 입력 이미지 정의
encoder_input = Input(shape = self.input_dim, name = "encoder_input")

x = encoder_input

for i in range(self.n_layers_encoder):
  conv_layer = Conv2D(
      filters = self.encoder_conv_filters[i],
      kernel_size = self.encoder_conv_kernel_size[i],
      strides = self.encoder_conv_strides[i],
      padding = 'same',
      name = 'encoder_conv_' + str(i)
  )

  x = conv_layer(x)
  x = LeakyReLU(x)

shape_before_flattening = K.int_shape(x)[1:]
x = Flatten()(x)

# Dense층으로 벡터를 2차원 잠재 공간으로 연결
encoder_output = Dense(self.z_dim, name = 'encoder_output')(x)
self.encoder = Model(encoder_input, encoder_output)

### 3.2.3 디코더
Autoencoder 클래스 중 디코더 부분

In [ ]:
# 디코더의 입력 즉, 잠개 공간의 포인트를 정의한다.
decoder_input = Input(shape = (self.z_dim,), name = 'decoder_input')

# 입력을 Dense층에 연결
x = Dense(np.prod(shape_before_flattening))(decoder_output)
# 전치 합성곱 층에 주입할 수 있도록 벡터를 다차원 텐서로 바꾼다.
x = Reshape(shape_before_flattening)(x)  

for i in range(self.n_layers_decoder):
  conv_t_layer = Conv2DTranspose(
      filters = self.decoder_conv_t_filters[i],
      kernel_size = self.decoder_conv_t_kernel_size[i],
      strides = self.decoder_conv_t_strides[i],
      padding = 'same',
      name = 'decoder_conv_t' + str(i)
  )

  x = conv_t_layer(i)

  if i < self.n_layers_decoder - 1:
    x = LeakyReLU(x)
  else:
    x = Activation('sigmoid')(x)

decoder_output = x

# 잠재 공간의 한 포인트를 받아 원본 이미지 차원으로 디코딩
self.decoder = Model(decoder_input, decoder_output)

### 3.2.4 인코더와 디코더 연결하기

In [17]:
model_input = encoder_input
model_output = decoder(encoder_output)
self.model = Model(model_input, model_output)

In [ ]:
### 컴파일
optimizer = Adam(lr = learning_rate)

def r_loss(y_true, y_pred):
  return K.mean(K.square(y_true - y_pred), axis = [1, 2, 3])

self.model.compile(optimizer = optimizer, loss = r_loss)

In [ ]:
self.model.fit(
    x_train, y_train, batch_size = batch_size, shuffle = True, epochs = 10, callbacks = callbacks_list
)

## 3.4 VAE 만들기
### 3.4.1 인코더
VAE 클래스 중 인코더 부분

In [ ]:
encoder_input = Input(shape = self.input_dim, name = 'encoder_input')

x = encoder_input

for i in range(self.n_layers_encoder):
  conv_layer = Conv2D(
      filters = self.encoder_conv_filters[i],
      kernel_size = self.encoder_conv_kernel_size[i],
      strides = self.encoder_conv_strides[i],
      padding = 'same',
      name = 'encoder_conv_' + str(i)
  )

  x = conv_layer(x)

  if self.use_batch_norm:
    x = BatchNormalization()(x)

  x = LeakyReLU()(x)
  if self.use_dropout:
    x = Dropout(rate = 0.25)(x)

shape_before_flattening = K.int_shape(x)[1:]
x = Flatten()(x)

# Flatten층을 바로 2차원 잠재 공간에 연결하지 않고 mu층과 log_var 층에 연결
self.mu = Dense(self.z_dim, name = 'mu')(x)
self.log_var = Dense(self.z_dim, name = 'log_var')(x)

encoder_mu_log_var = Model(encoder_input, (self.mu, self.log_var))

def sampling(args):
  mu, log_var = args
  epsilon = K.random_normal(shape = K.shape(mu), mean = 0, stddev = 1.)
  return mu + K.exp(log_var / 2) * epsilon

# Lambda 층이 잠재 공간에서 mu와 log_var로 정의되는 정규분포로부터 포인트 z를 샘플링한다.
encoder_output = Lambda(sampling, name = 'encoder_output')([self.mu, self.log_var])

# 이 모델은 입력 이미지를 받아 mu와 log_var로 정의된 정규 분포에서 한 포인트를 샘플링하여 2차원 잠재 공간으로 인코딩한다.
encoder = Model(encoder_input, encoder_output)